In [46]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.pipeline import Pipeline

In [2]:
def load_csv(features_path, labels_path):
    X = pd.read_csv(features_path, index_col=0)
    y = pd.read_csv(labels_path, index_col=0)
    return X, y

In [3]:
np.random.seed(0)
X, y = load_csv('data/train_features.csv', 'data/train_targets_scored.csv')
X['cp_type'] = X['cp_type'].map({'ctl_vehicle': 0, 'trt_cp': 1})
X['cp_dose'] = X['cp_dose'].map({'D1': 1, 'D2': 2})
X = X.to_numpy()
y = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
params = {'objective':'binary:logistic', 
          'n_estimators':2, 
          'random_state': 0,
          'eval_metric': 'logloss',
          'verbosity': 1}
model = MultiOutputClassifier(xgb.XGBClassifier(**params))
clf = Pipeline([('classify', model)])
print(clf)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               eval_metric='logloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                       

In [18]:
clf.fit(X_train, y_train)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               eval_metric='logloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                       

In [19]:
y_hat = clf.predict_proba(X_test)

In [21]:
predictions = np.array(y_hat)[:, :, 1].T

In [24]:
predictions.shape

(5954, 206)

In [25]:
y_test.shape

(5954, 206)

In [26]:
log_loss(y_test, predictions)

3.641420941165441

In [34]:
X, y = load_csv('data/train_features.csv', 'data/train_targets_scored.csv')
y = y[X['cp_type'] == 'trt_cp']
X = X[X['cp_type'] == 'trt_cp'].iloc[:, 1:]
X['cp_dose'] = X['cp_dose'].map({'D1': 1, 'D2': 2})
X = X.to_numpy()
y = y.to_numpy()
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y)

(21948, 874)
(21948, 206)


In [35]:
params = {'objective':'binary:logistic', 
          'n_estimators':2, 
          'random_state': 0,
          'eval_metric': 'logloss',
          'verbosity': 1}
model = MultiOutputClassifier(xgb.XGBClassifier(**params))
clf = Pipeline([('classify', model)])
print(clf)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               eval_metric='logloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                       

In [36]:
clf.fit(X_train, y_train)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               eval_metric='logloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                       

In [37]:
y_hat = clf.predict_proba(X_test)

In [38]:
predictions = np.array(y_hat)[:, :, 1].T
log_loss(y_test, predictions)

3.928871252985628

In [43]:
len(predictions[np.where(predictions < 0.5)])

1129459

In [45]:
predictions.size

1130322

In [49]:
params = {'min_child_weight': 1,
          'objective':'binary:logistic',
          'random_state': 0,
          'eval_metric': 'logloss',
          'verbosity': 1,
          'scale_pos_weight': 1,  # because of high class imbalance
}
grid_params = {'xgb__estimator__eta': [0.01, 0.05, 0.1, 0.2, 0.3],
               'xgb__estimator__max_depth': [4, 6, 8, 10],
               'xgb__estimator__gamma': [0, 0.05, 0.1, 0.3],
               'xgb__estimator__max_delta_step': [0, 0.5, 1, 2], 
               'xgb__estimator__colsample_bytree': [0.5, 0.7, 0.9, 1],
               'xgb__estimator__subsample': [0.5, 0.7, 0.9],
               'xgb__estimator__n_estimators': range(50, 400, 50)}
model = MultiOutputClassifier(xgb.XGBClassifier(**params))
xgb_pipeline = Pipeline([('xgb', model)])
clf = GridSearchCV(xgb_pipeline, grid_params)
print(clf)

GridSearchCV(estimator=Pipeline(steps=[('xgb',
                                        MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                                                      booster=None,
                                                                                      colsample_bylevel=None,
                                                                                      colsample_bynode=None,
                                                                                      colsample_bytree=None,
                                                                                      eval_metric='logloss',
                                                                                      gamma=None,
                                                                                      gpu_id=None,
                                                                                      importance_type='gain',
      

In [50]:
clf.fit(X_train, y_train)

KeyboardInterrupt: 